In [1]:
import json
import os

# Installing the Kaggle package
!pip install kaggle 

!mkdir /root/.kaggle/ 

#Important Note: complete this with your own key - after running this for the first time remmember to **remove** your API_KEY
api_token = {"username":"avielbbaz","key":"b9e35e8b71e0611fbd90b49b45fde304"}


# creating kaggle.json file with the personal API-Key details 
# You can also put this file on your Google Drive
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!kaggle datasets list -s marvel

ref                                                       title                                          size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  --------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
leonardopena/marvel-vs-dc                                 Marvel vs DC                                    2KB  2019-12-19 21:58:38           3247         64  1.0              
csanhueza/the-marvel-universe-social-network              The Marvel Universe Social Network              3MB  2017-01-28 11:28:49          16984        204  0.9705882        
deepcontractor/marvel-comic-books                         Marvel Comic Books Dataset                      4MB  2022-07-21 11:35:25           1065        105  1.0              
dannielr/marvel-superheroes                               Marvel Superheroes                              8MB  2018-07-2

In [ ]:
!mkdir ./datasets
!mkdir ./datasets/Marvel-Social-Network


In [ ]:
!chmod 600 /root/.kaggle/kaggle.json
# download the dataset from Kaggle and unzip it
!kaggle datasets download csanhueza/the-marvel-universe-social-network -p ./datasets/Marvel-Social-Network
!chdir ./datasets/Marvel-Social-Network
!unzip ./datasets/Marvel-Social-Network/*.zip  -d ./datasets/Marvel-Social-Network
!ls ./datasets/Marvel-Social-Network
# !chdir ./datasets/Marvel-Social-Network/database.sqlite/
# !unzip ./datasets/Marvel-Social-Network/database.sqlite/*.zip  -d ./datasets/Marvel-Social-Network/database.sqlite

  0% 0.00/2.72M [00:00<?, ?B/s]
100% 2.72M/2.72M [00:00<00:00, 191MB/s]
/bin/bash: chdir: command not found
Archive:  ./datasets/Marvel-Social-Network/the-marvel-universe-social-network.zip
  inflating: ./datasets/Marvel-Social-Network/edges.csv  
  inflating: ./datasets/Marvel-Social-Network/hero-network.csv  
  inflating: ./datasets/Marvel-Social-Network/nodes.csv  
edges.csv  hero-network.csv  nodes.csv	the-marvel-universe-social-network.zip


In [ ]:
## download most significant earthshakes database
!mkdir ./datasets/Earthshakes
!kaggle datasets download -d usgs/earthquake-database -p ./datasets/Earthshakes
!chdir ./datasets/Earthshakes
!unzip ./datasets/Earthshakes/*.zip  -d ./datasets/Earthshakes


  0% 0.00/590k [00:00<?, ?B/s]
100% 590k/590k [00:00<00:00, 108MB/s]
/bin/bash: chdir: command not found
Archive:  ./datasets/Earthshakes/earthquake-database.zip
  inflating: ./datasets/Earthshakes/database.csv  


In [2]:
!mkdir ./datasets/FIFA
!kaggle datasets download -d martj42/international-football-results-from-1872-to-2017 -p ./datasets/FIFA
!chdir ./datasets/FIFA
!unzip ./datasets/FIFA/*.zip  -d ./datasets/FIFA



mkdir: cannot create directory ‘./datasets/FIFA’: No such file or directory
  0% 0.00/700k [00:00<?, ?B/s]
100% 700k/700k [00:00<00:00, 82.6MB/s]
/bin/bash: chdir: command not found
Archive:  ./datasets/FIFA/international-football-results-from-1872-to-2017.zip
  inflating: ./datasets/FIFA/goalscorers.csv  
  inflating: ./datasets/FIFA/results.csv  
  inflating: ./datasets/FIFA/shootouts.csv  


In [3]:
import pandas as pd
import numpy as np

**Part 1 - Marvel Social Network**

In [ ]:


# Data Importing
hero_df = pd.read_csv('./datasets/Marvel-Social-Network/hero-network.csv',header=0)
edges_df = pd.read_csv('./datasets/Marvel-Social-Network/edges.csv',header=0)
nodes_df=pd.read_csv('./datasets/Marvel-Social-Network/nodes.csv',header=0)

Q1

In [ ]:
switch_df = hero_df.rename(columns={'hero1':'hero2','hero2':'hero1'})
friend_instances = hero_df.append(switch_df).drop_duplicates()
g = friend_instances.groupby('hero1').count()
g1 = g[g['hero2'] == 1]
len(g1)

53

Q2

In [ ]:
def find_heroes_friends(hero_name):
  heroes_friends = set(friend_instances[friend_instances['hero1'] == hero_name]['hero2'].values)
  return heroes_friends
spidermans_friends = find_heroes_friends('SPIDER-MAN/PETER PAR')
cap_friends = find_heroes_friends('CAPTAIN AMERICA')
iron_man_friends = find_heroes_friends('IRON MAN/TONY STARK ')
common_friends = spidermans_friends.intersection(cap_friends,iron_man_friends) ## intersection- returns a list/set that have the same people in it
len(common_friends)


700

Q3

In [ ]:
all_heros_df = edges_df['hero'].drop_duplicates()
length_heros = all_heros_df.str.len()
heros_length = pd.merge(all_heros_df,length_heros,left_index=True,right_index=True)
longest_n = heros_length[heros_length['hero_y'] == length_heros.max()]
shortest_n = heros_length[heros_length['hero_y'] == length_heros.min()]
print(f'The heroes whith the shortest names are: {list(shortest_n["hero_x"])}')
print(f'The heroes whith the longest names are: {list(longest_n["hero_x"])}')

The heroes whith the shortest names are: ['A', 'M', 'P', 'Q', 'T']
The heroes whith the longest names are: ['SPIDER-MAN/PETER PARKER']


Q4

In [ ]:
def heroes_friends_but_not_captains_friends(heroes_name):
  heroes_friends = find_heroes_friends(heroes_name)
  heroes_friends_without_captains_friends = heroes_friends.difference(cap_friends) ## difference- returns a list/set that are not in the object
  return len(heroes_friends_without_captains_friends)
heroes_friends_but_not_captains_friends('SPIDER-MAN/PETER PAR')

815

Q5

In [ ]:
spidermans_friends_ = set(hero_df[hero_df['hero1']=='SPIDER-MAN/PETER PAR'] ['hero2'].values)
spidermans_friends_ |= set(hero_df[hero_df['hero2']== 'SPIDER-MAN/PETER PAR'] ['hero1'].values)



friends_of_spideys_friends = set()

for friend in spidermans_friends_:
  friends_of_spideys_friends |= set(hero_df[hero_df['hero1'] == friend] ['hero2'].values)
  friends_of_spideys_friends |= set(hero_df[hero_df['hero2'] == friend] ['hero1'].values)

len(friends_of_spideys_friends)
print(f"the total number of friends of spideys friends is:{len(friends_of_spideys_friends)}")

the total number of friends of spideys friends is:6355


**Part 2 - Earthquake**

In [ ]:
earthquake_df = pd.read_csv('/content/datasets/Earthshakes/database.csv')
earthquake_df

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
0,01/02/1965,13:44:18,19.2460,145.6160,Earthquake,131.60,NaN,NaN,6.0,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic
1,01/04/1965,11:29:49,1.8630,127.3520,Earthquake,80.00,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic
2,01/05/1965,18:05:58,-20.5790,-173.9720,Earthquake,20.00,NaN,NaN,6.2,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic
3,01/08/1965,18:49:43,-59.0760,-23.5570,Earthquake,15.00,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic
4,01/09/1965,13:32:50,11.9380,126.4270,Earthquake,15.00,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23407,12/28/2016,08:22:12,38.3917,-118.8941,Earthquake,12.30,1.2,40.0,5.6,ML,...,18.0,42.47,0.120,NaN,0.1898,NN00570710,NN,NN,NN,Reviewed
23408,12/28/2016,09:13:47,38.3777,-118.8957,Earthquake,8.80,2.0,33.0,5.5,ML,...,18.0,48.58,0.129,NaN,0.2187,NN00570744,NN,NN,NN,Reviewed
23409,12/28/2016,12:38:51,36.9179,140.4262,Earthquake,10.00,1.8,NaN,5.9,MWW,...,NaN,91.00,0.992,4.8,1.5200,US10007NAF,US,US,US,Reviewed
23410,12/29/2016,22:30:19,-9.0283,118.6639,Earthquake,79.00,1.8,NaN,6.3,MWW,...,NaN,26.00,3.553,6.0,1.4300,US10007NL0,US,US,US,Reviewed


Q1

In [ ]:
highest_magnitude = earthquake_df[earthquake_df['Magnitude']==earthquake_df['Magnitude'].max()]
highest_magnitude

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
17083,12/26/2004,00:58:53,3.295,95.982,Earthquake,30.0,NaN,601.0,9.1,MW,...,NaN,22.0,NaN,NaN,1.17,OFFICIAL20041226005853450_30,OFFICIAL,US,OFFICIAL,Reviewed
20501,03/11/2011,05:46:24,38.297,142.373,Earthquake,29.0,NaN,541.0,9.1,MWW,...,NaN,9.5,NaN,NaN,1.16,OFFICIAL20110311054624120_30,OFFICIAL,US,OFFICIAL,Reviewed


Q2

In [ ]:
median_magnitude_nuc = earthquake_df[earthquake_df['Type']=='Nuclear Explosion'] ['Magnitude'].median()
print(f'the median magnitude caused by nuclear explosion is {median_magnitude_nuc}')

the median magnitude caused by nuclear explosion is 5.9


Q3

In [ ]:
def max_number_earthquakes_by_year(number_magnitude,earthquake_df):
  earthquake_df['year'] = earthquake_df['Date'].str.slice(start=6)
  earthquake_df = earthquake_df[earthquake_df['Magnitude']>number_magnitude] [['year','Magnitude']]
  group_year = earthquake_df.groupby(['year'],as_index=False).count()
  year_with_max_earthquakes = group_year[group_year['Magnitude']==group_year['Magnitude'].max()] 
  return year_with_max_earthquakes['year'].values
print(f'the year with the most earthquakes above magnitude 5: {max_number_earthquakes_by_year(5,earthquake_df)}')
print(f'the years with the most earthquakes above magnitude 8: {max_number_earthquakes_by_year(8,earthquake_df)}')

the year with the most earthquakes above magnitude 5: ['2011']
the years with the most earthquakes above magnitude 8: ['2007']


Q4

In [ ]:
earthquake_df['hour'] = earthquake_df['Time'].str[:2]
most_dangerous_hour = earthquake_df['hour'].mode().values ## mode- finds the most common value from series (numpy array)
print(f'the most dangerous hour is: {int(most_dangerous_hour)}:00-{int(most_dangerous_hour)+1}:00')

the most dangerous hour is: 13:00-14:00


**Part 3 - Fifa 19**

In [4]:
goalscorers_df = pd.read_csv('/content/datasets/FIFA/goalscorers.csv')
results_df = pd.read_csv('/content/datasets/FIFA/results.csv')
shootouts_df = pd.read_csv('/content/datasets/FIFA/shootouts.csv')

In [5]:
results_df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False


Q1

In [7]:
def get_weak_team(team_name):
  results_wanted_away_team_df = results_df[results_df['away_team'] == team_name]
  g_home_away = results_wanted_away_team_df.groupby('home_team',as_index=False).sum()
  highest_away_score = g_home_away[g_home_away['away_score']==g_home_away['away_score'].max()]
  highest_away_score = highest_away_score.sort_values(by = ['home_team']).iloc[0]
  return highest_away_score['home_team']
print(f"the weekest team against Italy's team is: {get_weak_team('Italy')}")
print(f"the weekest team against Israel's team is: {get_weak_team('Israel')}")
print(f"the weekest team against Argentina's team is: {get_weak_team('Argentina')}")

the weekest team against Italy's team is: Switzerland
the weekest team against Israel's team is: Luxembourg
the weekest team against Argentina's team is: Uruguay


Q2

In [13]:
def get_score_dates(team1,team2):
  result_team1_team2_df = results_df[results_df['home_team'].isin([team1,team2])]
  result_team1_team2_df = result_team1_team2_df[result_team1_team2_df['away_team'].isin([team1,team2])]
  if len(result_team1_team2_df) == 0:
    return '2022-21-11','2022-21-11'
  result_team1_team2_df['total_score'] = result_team1_team2_df['away_score'].values + result_team1_team2_df['home_score'].values
  max_score_date = result_team1_team2_df[result_team1_team2_df['total_score']==result_team1_team2_df['total_score'].max()] ['date']
  max_score_date = max_score_date.sort_values().iloc[-1]
  min_score_date = result_team1_team2_df[result_team1_team2_df['total_score']==result_team1_team2_df['total_score'].min()] ['date']
  min_score_date = min_score_date.sort_values().iloc[-1]
  return max_score_date , min_score_date
print(f"Argenrina-Brazil: dates of games with highest score: {get_score_dates('Argentina','Brazil')[0]},\ndates of games with lowest scores: {get_score_dates('Argentina','Brazil')[1]} ")
print(f"Cameroon-England: dates of games with highest score: {get_score_dates('Cameroon','England')[0]},\ndates of games with lowest scores: {get_score_dates('Cameroon','England')[1]} ")
print(f"Israel-Iran: dates of games with highest score: {get_score_dates('Israel','Iran')[0]},\ndates of games with lowest scores: {get_score_dates('Israel','Iran')[1]} ")

Argenrina-Brazil: dates of games with highest score: 1945-12-20,
dates of games with lowest scores: 2021-11-16 
Cameroon-England: dates of games with highest score: 1990-07-01,
dates of games with lowest scores: 1997-11-15 
Israel-Iran: dates of games with highest score: 1968-05-19,
dates of games with lowest scores: 1959-12-12 


Q3

In [ ]:
new_results_df = results_df
new_results_df['total_score'] = results_df[['home_score','away_score']].sum(axis=1)
g_tournament = new_results_df.groupby('tournament',as_index=False).mean()
highest_average_score_tour = g_tournament[g_tournament['total_score']==g_tournament['total_score'].max()] ['tournament'].values
lowest_average_score_tour = g_tournament[g_tournament['total_score']==g_tournament['total_score'].min()] ['tournament'].values
print(f"the tournament with the highest average score is: {highest_average_score_tour}")
print(f"the tournament with the lowest average score is: {lowest_average_score_tour}")

the tournament with the highest average score is: ['Pacific Games']
the tournament with the lowest average score is: ['FIFA 75th Anniversary Cup']
